In [ ]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.cluster import KMeansClusterer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import json
from nltk.stem import WordNetLemmatizer 
%matplotlib inline

# 1. Read articles from text file

In [ ]:
articles = None
filename = 'YOUR_FILE_NAME.txt'
folder_name = 'YOUR_FOLDER_NAME'
with open(folder_name + '/' + filename) as json_file:  
    articles = json.load(json_file)
df = pd.DataFrame(articles)

# 2. Text processing

In [ ]:
lemmatizer = WordNetLemmatizer()
def process_text(text):
    words = nltk.word_tokenize(text)
    words = [w.lower() for w in words]
    words = [w.translate(str.maketrans('', '', string.punctuation)) for w in words]
    words = [w for w in words if w not in nltk.corpus.stopwords.words('english')]
    words = [lemmatizer.lemmatize(w) for w in words]
    words = [w for w in words if w.isalnum()] 
    new_text = " ".join(words)
    return new_text

# 2.a Process "body" of the article

In [ ]:
df['body'] = [process_text(b) for b in df['body']]

# (Optional) Split articles (i.e. old articles vs new article)

In [ ]:
# df_new =df.iloc[:1,:]
# df_old = df.iloc[1:,:]